In [1]:
# Import necessary libraries
from datetime import date
import os
import urllib.request
from urllib.request import urlopen
import mechanicalsoup
from bs4 import BeautifulSoup as bs
import time
import random

# Check Validity of URL link

In [2]:
def check_validity(my_url):
    try:
      header={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
      req = urllib.request.Request(url=my_url, headers=header) 
      # page = urllib.request.urlopen(req).read()
      urlopen(req).read()
      print("Valid URL")
    except IOError:
        print ("Invalid URL")
        sys.exit()


url2dd = 'https://www.testpapersfree.com/' # 403 error3
url2dd1 = "https://www.sgtestpaper.com/" # 406 error (not free papers)
url2dd2 = "https://sgexam.com/"
url2dd0 = 'https://form.select.com' # gibberish url
check_validity(url2dd)

Valid URL


# Testpapersfree.com

## Step 1: Get the list of subjects and year for phase

In [3]:
def getparams_jc():
    url = "https://testpapersfree.com/"
    phase = "junior-college"
    url2dd = url + phase

    # Create list of levels
    level = ['A-level', 'JC1']

    import urllib
    #defining header
    header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
        'AppleWebKit/537.11 (KHTML, like Gecko) '
        'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

    #the URL where you are requesting at
    req = urllib.request.Request(url=url2dd, headers=header)
    page = urllib.request.urlopen(req).read()

    from bs4 import BeautifulSoup as bs
    html_page = bs(page, "html.parser")
    # print (html_page)

    # gets all params
    params = []
    for l in html_page.select("div")[-1].find_all("a"):
        curr_l = l.get("href")
        if curr_l.startswith("/"+phase):
            params.append(curr_l)

    # return index number
    for i, ele in enumerate(params):
        split_ele = ele.split("/")[2]
        # print (i, split_ele)
        if split_ele == "prelims":          # check and change
            ind_p = i
        elif split_ele == "2016":           # check and change the oldest year
            ind = i         # returns index number when oldest available year is reached
    
    sub = []
    year = []
    for i, ele in enumerate(params):
        split_ele = ele.split("/")[2]
        if i > ind:
            sub.append(split_ele)
        elif i > ind_p:
            year.append(split_ele)
    # print (sec_year)
    # print (sec_sub)
    return level, year, sub

getparams_jc()

(['A-level', 'JC1'],
 ['2018', '2017', '2016'],
 ['biology-h1',
  'biology-h2',
  'chemistry-h1',
  'chemistry-h2',
  'economics-h1',
  'economics-h2',
  'geography-h1',
  'geography-h2',
  'gp-h1',
  'gp-h2',
  'history-h1',
  'history-h2',
  'maths-h1',
  'maths-h2',
  'physics-h1',
  'physics-h2'])

---

## Step 2: Define function for downloading pdf

In [4]:
# Custom function for downloading pdfs
def getpdfjc(sel_sub:str, sel_level:str, sel_year:str):

    url = "https://testpapersfree.com/"
    phase = "junior-college"
    url2dd = url + phase

    ###################### 0th layer Get year, sub list ######################
    ###################### 0th layer Get year, sub list ######################
    ###################### 0th layer Get year, sub list ######################
    level_list, year_list, sub_list = getparams_jc()

    # SELECTION ERROR CHECKS
    # Subject
    if sel_sub in sub_list:    
        sel_url = url2dd + '/' + sel_sub + '/'
    else:
        raise AttributeError(f'Please input correct subject from {sub_list}')

    # Year
    if sel_year in year_list:
        pass
    else:
        raise AttributeError(f'Please input correct year from {year_list}')

    # Level
    if sel_level in level_list:
        pass
    else:
        raise AttributeError(f'Please input correct level from {level_list}')

    ###################### 1st layer Form Interaction ######################
    ###################### 1st layer Form Interaction ######################
    ###################### 1st layer Form Interaction ######################

    # Now we need to interact with website to fill out forms 
    # we use mechanicalsoup library

    browser = mechanicalsoup.Browser(soup_config={'features': 'lxml'})
    # Browser objects represent the headless web browser. You can use them to request a page from the Internet by passing a URL to their .get() method:

    page = browser.get(sel_url)
    # page is a Response object that stores the response from requesting the URL from the browser:
    # The number 200 represents the status code returned by the request. 
    # A status code of 200 means that the request was successful. 
    # An unsuccessful request might show a status code of 404 if the URL doesn’t exist or 500 if there’s a server error when making the request.

    # view the HTML by inspecting the .soup attribute:
    if str(page) == "<Response [200]>":
        html1 = page.soup
    else:
        print ("Browser main url error")

    # Access everything inside the <form> tags 
    form = html1.select("form")[0]

    # ********************************************
    # ********************************************
    # ********************************************


    # Variables for selecting filters
    form.select("select")[0].select('option')[0]['value'] = sel_level        # CAN CHANGE VALUES HERE
    form.select("select")[1].select('option')[0]['value'] = sel_year       # CAN CHANGE VALUES HERE
    print (sel_year, sel_level, sel_sub)


    ###################### 2nd layer Form Interaction ######################
    ###################### 2nd layer Form Interaction ######################
    ###################### 2nd layer Form Interaction ######################

    # Generate the url link after form is selected (1st page) 
    selected1_page = browser.submit(form, page.url)

    # Turn 1st page into html doc in order to look for links to page 2,3 etc.
    sel1_page = browser.get(selected1_page.url)
    if str(sel1_page) == '<Response [200]>':
        sel1_html = sel1_page.soup
        # Look for links to extra pages and append to a list
        html_pages_links=[selected1_page.url]
        for l in sel1_html.select("table")[-1].find_all("a"):
            cur_l = l.get("href")
            if "page" in cur_l:
                pagelink = url+cur_l.replace(" ","%20",2)[1:]
                if pagelink not in html_pages_links:
                    html_pages_links.append(pagelink)
    else:
        print ("browser url error")

    ###################### Get links to each paper's pdf download page ######################
    ###################### Get links to each paper's pdf download page ######################
    ###################### Get links to each paper's pdf download page ######################

    # create empty list to append the individual links to pdf
    pdf_pagelinks = []

    # Loop through the html_pages_links in order to extract the invidiual links to the pdf site
    for page in html_pages_links:
        sel_page = browser.get(page)
        if str(sel_page) == "<Response [200]>":
            sel_html = sel_page.soup
            time.sleep(random.randint(1, 3))
            # Access everything inside the <form> tags
            for i in range(1, len(sel_html.find_all("form"))):
                ind_link = url + str(sel_html.select("form")[i]).split('"')[1][1:]
                pdf_pagelinks.append(ind_link)   
        else:
            print ("browser url error")


    ###################### Get pdf links ######################
    ###################### Get pdf links ######################
    ###################### Get pdf links ######################

    # Generate pdf links 
    header={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
    'AppleWebKit/537.11 (KHTML, like Gecko) '
    'Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'}

    pdf_links=[]
    for pagelink in pdf_pagelinks:
        req = urllib.request.Request(url=pagelink, headers=header)
        page = urllib.request.urlopen(req).read()
        html_page = bs(page, "html.parser")
        for pdflink in html_page.find_all("a"):
            cur_link = pdflink.get("href")
            if cur_link.endswith("pdf"):
                pdf_links.append(url+cur_link.replace(" ","%20")[3:])           
        time.sleep(random.randint(1, 3))

    ###################### Download pdf links ######################
    ###################### Download pdf links ######################
    ###################### Download pdf links ######################

    # Download pdf section
    count=0
    for dldlink in pdf_links:
        req = urllib.request.Request(url=dldlink, headers=header)
        req1 = urllib.request.urlopen(req, timeout=10).read()
        f_name = dldlink.split("/")[-1]

        if f_name.count("-")>1:
            sel_type = None
            for i in f_name.split("-")[:6]:
                if len(i) == 3 and i[1]=="A":
                    sel_type = str(i) + '/'
            if sel_type is not None:
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/' + sel_type
            else:
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/'
        elif f_name.count("_")>1:
            sel_type_j = None
            for j in f_name.split("_")[:6]:
                if len(j) == 3 and j[1]=="A":
                    sel_type_j = str (j) + '/'
            if sel_type_j is not None :
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/' + sel_type_j
            else:
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/'            
        else:
            sel_type_k = None
            for k in f_name.split(" ")[:6]:
                if len(k) == 3 and k[1]=="A":
                    sel_type_k = str (k) + '/'
            if sel_type_k is not None :
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/' + sel_type_k
            else:
                path_dir = "dld_pdf/" + phase + '/' + sel_sub + '/' + sel_year + '/' + sel_level + '/'
        print (path_dir, f_name)
        if not os.path.exists(path_dir):
            os.makedirs(path_dir)
        try:
            # Download pdf to respective directory
            with open(f"{path_dir}" + f_name, 'wb')as file:
                file.write(req1)
            count += 1
            time.sleep(random.randint(5, 10))
        except:
            print ("\n Download Failed \n")
    if count == 0:
        print ("No papers to download", '\n')
    else:
        print ("Completed all downloads!", count, 'pdf files', '\n')

---

# Download pdf

In [5]:
level_list, year_list, sub_list = getparams_jc()

print (level_list)
print (year_list)
print (len(sub_list), sub_list)

['A-level', 'JC1']
['2018', '2017', '2016']
16 ['biology-h1', 'biology-h2', 'chemistry-h1', 'chemistry-h2', 'economics-h1', 'economics-h2', 'geography-h1', 'geography-h2', 'gp-h1', 'gp-h2', 'history-h1', 'history-h2', 'maths-h1', 'maths-h2', 'physics-h1', 'physics-h2']


In [21]:
# 1. biology-h1
for y in year_list:
    for l in level_list:
        getpdfjc("biology-h1", l, y)

2018 A-level biology-h1
dld_pdf/junior-college/biology-h1/2018/A-level/ ACJC_H1_Biology_Paper1.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ ACJC_H1_Biology_Paper2.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ DHS_H1_Biology_Paper2.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ NYJC_H1_Biology_Paper2.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ RI_H1_Biology_Paper1-and-2.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ VJC_H1_Biology_Paper2.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ NYJC_H1_Biology_Paper1.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ DHS_H1_BIO_P1.pdf
dld_pdf/junior-college/biology-h1/2018/A-level/ VJC_H1_BIO_Paper1.pdf
Completed all downloads! 9 pdf files 

2018 JC1 biology-h1
No papers to download 

2017 A-level biology-h1
dld_pdf/junior-college/biology-h1/2017/A-level/ y17_jc2_Biology_h1_prelim_Anderson.pdf
dld_pdf/junior-college/biology-h1/2017/A-level/ y17_jc2_Biology_h1_prelim_Anglo_Chinese.pdf
dld_pdf/junior-college/biol

In [23]:
# 2. biology-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[1], l, y)

2018 A-level biology-h2
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Anglo_Chinese.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Hwa_Chong.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Innova.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Jurong.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Millenia.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_National.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Raffles.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Serangoon.pdf
dld_pdf/junior-college/biology-h2/2018/A-level/ JC2_Biology_H2_2018_Victoria.pdf
Completed all downloads! 9 pdf files 

2018 JC1 biology-h2
No papers to download 

2017 A-level biology-h2
dld_pdf/junior-college/biology-h2/2017/A-level/ y17_jc2_Biology_h2_prelim_Anderson.pdf
dld_pdf/junior-college/biology-h2/2017/A-level/ y17

In [24]:
# 3. chemistry-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[2], l, y)

2018 A-level chemistry-h1
dld_pdf/junior-college/chemistry-h1/2018/A-level/ ACJC_H1_Chemistry_Paper1.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ NJC_H1_Chemistry_Paper1.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ NYJC_H1_Chemistry_Paper1.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ RI_H1_Chemistry_Paper1-and-2.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ ACJC_H1_Chemistry_Paper2.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ NJC_H1_Chemistry_Paper2.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ NYJC_H1_Chemistry_Paper2.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ VJC_H1_Chemistry_Paper1.pdf
dld_pdf/junior-college/chemistry-h1/2018/A-level/ VJC_H1_Chemistry_Paper2.pdf
Completed all downloads! 9 pdf files 

2018 JC1 chemistry-h1
No papers to download 

2017 A-level chemistry-h1
dld_pdf/junior-college/chemistry-h1/2017/A-level/ JC2_Chemistry_H1_2017_Anderson.pdf
dld_pdf/junior-college/chemistry-h1/2017/A-level/ JC2_Chemistry_H1_

In [25]:
# 4. chemistry-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[3], l, y)

2018 A-level chemistry-h2
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Anglo_Chinese.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Catholic.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Dunman.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Hwa_Chong.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Innova.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Jurong.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Meridian.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Millenia.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Nanyang.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_National.pdf
dld_pdf/junior-college/chemistry-h2/2018/A-level/ JC2_Chemistry_H2_2018_Pioneer.pdf
dld_pdf/junior-college/chemistry-h2/2018/

In [26]:
# 5. economics-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[4], l, y)

2018 A-level economics-h1
dld_pdf/junior-college/economics-h1/2018/A-level/ JC2_Econ_H1_2018_Hwa_Chong.pdf
dld_pdf/junior-college/economics-h1/2018/A-level/ JC2_Econ_H1_2018_Nanyang.pdf
dld_pdf/junior-college/economics-h1/2018/A-level/ JC2_Econ_H1_2018_National.pdf
dld_pdf/junior-college/economics-h1/2018/A-level/ JC2_Econ_H1_2018_Raffles.pdf
dld_pdf/junior-college/economics-h1/2018/A-level/ JC2_Econ_H1_2018_Victoria.pdf
Completed all downloads! 5 pdf files 

2018 JC1 economics-h1
No papers to download 

2017 A-level economics-h1
dld_pdf/junior-college/economics-h1/2017/A-level/ y17_jc2_Economics_h1_prelim_Anderson.pdf
dld_pdf/junior-college/economics-h1/2017/A-level/ y17_jc2_Economics_h1_prelim_Anglo_Chinese.pdf
dld_pdf/junior-college/economics-h1/2017/A-level/ y17_jc2_Economics_h1_prelim_Catholic.pdf
dld_pdf/junior-college/economics-h1/2017/A-level/ y17_jc2_Economics_h1_prelim_Dunman.pdf
dld_pdf/junior-college/economics-h1/2017/A-level/ y17_jc2_Economics_h1_prelim_Hwa_Chong.pdf
dld_p

In [27]:
# 6. economics-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[5], l, y)

2018 A-level economics-h2
dld_pdf/junior-college/economics-h2/2018/A-level/ HCI_H2_Economics_CSQs.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ HCI_H2_Economics_Essays.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NJC_H2_Economics_Paper2.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NJC_H2_Economics_Paper1.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ VJC_H2_Economics_Paper1.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ VJC_H2_Economics_Paper2.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NYJC_H2_Economics_Paper2.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NYJC_H2_ECONS_CSQ1_QP_with_answers.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NYJC_H2_ECONS_CSQ2_QP_with_answers.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ NYJC_H2_ECONS_P1.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ RI_H2_ECONS_P1.pdf
dld_pdf/junior-college/economics-h2/2018/A-level/ RI_H2_ECONS_P2.pdf
Completed all downloads! 12 pdf files 

20

In [28]:
# 7. geography-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[6], l, y)

2018 A-level geography-h1
No papers to download 

2018 JC1 geography-h1
No papers to download 

2017 A-level geography-h1
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Anglo_Chinese.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Catholic.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Dunman.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Hwa_Chong.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Innova.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Jurong.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Meridian.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Millenia.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y17_jc2_Geography__h1_prelim_Nanyang.pdf
dld_pdf/junior-college/geography-h1/2017/A-level/ y

In [6]:
# 8. geography-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[7], l, y)

2018 A-level geography-h2
dld_pdf/junior-college/geography-h2/2018/A-level/ NYJC_H2_Geography_Paper2.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ NYJC_H2_Geography_Paper1.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ VJC_H2_Geography_Pape1.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ VJC_H2_Geography_Paper2.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ RI_H2_Geography_Paper2.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ NJC_H2_GEO_Paper2.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ ACJC_H2_GEO_Paper2.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ ACJC_H2_GEO_P1_QP_Essay.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ NJC_H2_GEO_Paper1.pdf
dld_pdf/junior-college/geography-h2/2018/A-level/ RI_H2_Geography_Paper1.pdf
Completed all downloads! 10 pdf files 

2018 JC1 geography-h2
No papers to download 

2017 A-level geography-h2
No papers to download 

2017 JC1 geography-h2
No papers to download 

2016 A-level geography-h2
No p

In [7]:
# 9. gp-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[8], l, y)

2018 A-level gp-h1
dld_pdf/junior-college/gp-h1/2018/A-level/ JC2_GP_H1_2018_Catholic.pdf
dld_pdf/junior-college/gp-h1/2018/A-level/ JC2_GP_H1_2018_Temasek.pdf
dld_pdf/junior-college/gp-h1/2018/A-level/ JC2_GP_H1_2018_Victoria.pdf
Completed all downloads! 3 pdf files 

2018 JC1 gp-h1
No papers to download 

2017 A-level gp-h1
No papers to download 

2017 JC1 gp-h1
No papers to download 

2016 A-level gp-h1
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_ACJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_AJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_IJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_JJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_MJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_NJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_NYJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_PJC.pdf
dld_pdf/junior-college/gp-h1/2016/A-level/ JC2_GP_H1_2016_RV

In [8]:
# 10. gp-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[9], l, y)

2018 A-level gp-h2
No papers to download 

2018 JC1 gp-h2
No papers to download 

2017 A-level gp-h2
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Anderson.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Catholic.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Dunman.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Hwa_Chong.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Innova.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Jurong.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Meridian.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Millenia.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Nanyang.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_National.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Pioneer.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_2017_Raffles.pdf
dld_pdf/junior-college/gp-h2/2017/A-level/ JC2_GP_H2_20

In [9]:
# 11. history-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[10], l, y)

2018 A-level history-h1
No papers to download 

2018 JC1 history-h1
No papers to download 

2017 A-level history-h1
No papers to download 

2017 JC1 history-h1
No papers to download 

2016 A-level history-h1
No papers to download 

2016 JC1 history-h1
No papers to download 



In [10]:
# 12. history-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[11], l, y)

2018 A-level history-h2
dld_pdf/junior-college/history-h2/2018/A-level/ HCI_H2_HIST_9752_P1.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ HCI_H2_HIST_9752_P2.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ NJC_H2_HIST_P1_QP.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ NJC_H2_HIST_P2_QP.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ NYJC_H2_HIST_9752_P1.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ NYJC_H2_HIST_9752_P2.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ RI_H2_HIST_P1_Questions.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ RI_H2_HIST_P2_Questions.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ VJC_H2_HIST_P1.pdf
dld_pdf/junior-college/history-h2/2018/A-level/ VJC_H2_HIST_P2.pdf
Completed all downloads! 10 pdf files 

2018 JC1 history-h2
No papers to download 

2017 A-level history-h2
No papers to download 

2017 JC1 history-h2
No papers to download 

2016 A-level history-h2
No papers to download 

2016 JC1 history-h2
No papers to dow

In [11]:
# 13. maths-h1
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[12], l, y)

2018 A-level maths-h1
dld_pdf/junior-college/maths-h1/2018/A-level/ ACJC_H1_Maths.pdf
dld_pdf/junior-college/maths-h1/2018/A-level/ NJC_H1_Maths.pdf
dld_pdf/junior-college/maths-h1/2018/A-level/ NYJC_H1_Maths.pdf
dld_pdf/junior-college/maths-h1/2018/A-level/ VJC_H1_Maths.pdf
dld_pdf/junior-college/maths-h1/2018/A-level/ DHS_H1_MATH_QP.pdf
dld_pdf/junior-college/maths-h1/2018/A-level/ RI_H1_MATH_QP.pdf
Completed all downloads! 6 pdf files 

2018 JC1 maths-h1
No papers to download 

2017 A-level maths-h1
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Anderson.pdf
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Anglo_Chinese.pdf
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Dunman.pdf
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Hwa_Chong.pdf
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Innova.pdf
dld_pdf/junior-college/maths-h1/2017/A-level/ JC2_Maths_H1_2017_Jurong.pdf
dld_pdf/junior-college/maths-h

In [12]:
# 14. maths-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[13], l, y)

2018 A-level maths-h2
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Eunoia.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Anderson.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Anglo_Chinese.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Catholic.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Hwa_Chong.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Innova.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Jurong.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Meridian.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Millenia.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Nanyang.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_National.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_H2_2018_Pioneer.pdf
dld_pdf/junior-college/maths-h2/2018/A-level/ JC2_Maths_

In [13]:
# 15. maths-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[14], l, y)

2018 A-level physics-h1
No papers to download 

2018 JC1 physics-h1
No papers to download 

2017 A-level physics-h1
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Anderson.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Anglo_Chinese.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Dunman.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Hwa_Chong.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Innova.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Jurong.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Meridian.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Millenia.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Nanyang.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_National.pdf
dld_pdf/junior-college/physics-h1/2017/A-level/ JC2_Physics_H1_2017_Pioneer

In [14]:
# 16. maths-h2
for y in year_list:
    for l in level_list:
        getpdfjc(sub_list[15], l, y)

2018 A-level physics-h2
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Anderson.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Anglo_Chinese.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Catholic.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Dunman.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Eunoia.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Hwa_Chong.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Innova.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Jurong.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Meridian.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Millenia.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_Nanyang.pdf
dld_pdf/junior-college/physics-h2/2018/A-level/ JC2_Physics_H2_2018_National.pdf
dld_pdf